In [2]:
# Imports
import tifffile
import numpy as np
from scripts.config_model import save_experiment_config, tree
from scripts.sample_db import SampleDB
from scripts.utils.image_utils import load_tiff_as_hyperstack, save_array_as_hyperstack_tiff
import os



In [3]:
# Step 1: Load the sample database
db_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'
sample_db = SampleDB()
sample_db.load(db_path)
print(sample_db)

SampleDB(sample_ids=['20220426_RM0008_130hpf_fP1_f3', '20220118_RM0012_124hpf_fP8_f2', '20220427_RM0008_126hpf_fP3_f3'])


In [4]:
# Step 2: Load experiment configuration
sample_id = '20220118_RM0012_124hpf_fP8_f2'
exp = sample_db.get_sample(sample_id)

In [5]:
# Step 3: Making shortcuts of sample parameters/information
sample = exp.sample
root_path = exp.paths.root_path
trials_path = exp.paths.trials_path
anatomy_path = exp.paths.anatomy_path
em_path = exp.paths.em_path
n_planes = exp.params_lm.n_planes
n_frames = exp.params_lm.n_frames
n_slices = exp.params_lm.lm_stack_range
doubling = 2 if exp.params_lm.doubling else 1

# Calculating number of frames per trial 
n_frames_trial = n_frames // n_planes
exp.params_lm["n_frames_trial"] = n_frames_trial

# Getting paths of the trial acquisitions
raw_trial_paths = os.listdir(os.path.join(trials_path, "raw"))
n_trials = len(raw_trial_paths)
exp.params_lm["n_trials"] = n_trials
ignore_until_frame = exp.params_lm.shutter_delay_frames  # edit if you want to avoid summing out motor movements at the beginning of acquisition

# Define the path for the preprocessed folder
processed_folder = os.path.join(trials_path, "processed")
os.makedirs(processed_folder, exist_ok=True)

ref_images_path = os.path.join(processed_folder, f"sum_raw_trials_{sample.id}.tif")

In [7]:
# Load planes and stack
for trial_path in raw_trial_paths:
    trial_images = load_tiff_as_hyperstack(os.path.join(trials_path, "raw", trial_path))



\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-01-18\f2\trials\raw\20220118_RM0012_124hpf_fP8_f2_t1_o1Ala_001_.tif loaded.
\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-01-18\f2\trials\raw\20220118_RM0012_124hpf_fP8_f2_t1_o2Ser_001_.tif loaded.
\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-01-18\f2\trials\raw\20220118_RM0012_124hpf_fP8_f2_t1_o3TDCA_001_.tif loaded.
\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-01-18\f2\trials\raw\20220118_RM0012_124hpf_fP8_f2_t1_o4Ctrl_001_.tif loaded.
\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-01-18\f2\trials\raw\20220118_RM0012_124hpf_fP8_f2_t1_o5GCA_001_.tif loaded.


KeyboardInterrupt: 